# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import sys
import json

import nbformat

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import mbrain as mb

# Jupyter

In [4]:
with open('Example.ipynb', 'r') as f:
    nb = nbformat.read(f, as_version=4)

In [5]:
cell = nb['cells'][2]

In [6]:
mb.is_flashcard(nb['cells'][2])

True

In [7]:
print(cell.source)

<!-- {"id": "1560026882572"} -->

**Example question goes here**

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs <span>\$</span>10 and <span>\$</span>20 usd
* bullet with $x = 4$ math and $x = 5$ more math

$$ x = 2^4 $$

$$ y = 1^3 $$

```python
def example_code():
    return 'hohoho'
```



In [8]:
meta, head, body = mb.process_cell_source(cell.source)
print('META:\n', meta, '\n')
print('HEAD:\n', head, '\n')
print('BODY:\n', body)

META:
 {'id': '1560026882572'} 

HEAD:
 Example question goes here 

BODY:
 
<div class="cell border-box-sizing text_cell rendered"><div class="prompt input_prompt">
</div><div class="inner_cell">
<div class="text_cell_render border-box-sizing rendered_html">
<p>Some good and clear answer goes here</p>
<ul>
<li>maybe a bullet point</li>
<li>or make it two</li>
<li>add some dollar signs $10 and $20 usd</li>
<li>bullet with \(x = 4\) math and \(x = 5\) more math</li>
</ul>
\[ x = 2^4 \]\[ y = 1^3 \]<div class="highlight"><pre><span></span><span class="k">def</span> <span class="nf">example_code</span><span class="p">():</span>
    <span class="k">return</span> <span class="s1">&#39;hohoho&#39;</span>
</pre></div>

</div>
</div>
</div>
 




In [9]:
new_source = mb.put_meta(cell.source, anki_id='1234567890')
print(new_source)

<!-- {"id": "1234567890"} -->

**Example question goes here**

Some good and clear answer goes here

* maybe a bullet point
* or make it two
* add some dollar signs <span>\$</span>10 and <span>\$</span>20 usd
* bullet with $x = 4$ math and $x = 5$ more math

$$ x = 2^4 $$

$$ y = 1^3 $$

```python
def example_code():
    return 'hohoho'
```



In [10]:
cell.source = new_source

In [11]:
with open('Example_Out.ipynb', 'w') as f:
    nbformat.write(nb, f)

In [12]:
!diff Example.ipynb Example_Out.ipynb

21c21
<     "<!-- {\"id\": \"1560026882572\"} -->\n",
---
>     "<!-- {\"id\": \"1234567890\"} -->\n",


In [13]:
from IPython.core.display import HTML

In [14]:
display(HTML(body))

# Anki Connect

In [15]:
mb.anki_test_db()

In [16]:
mb.anki_get_decks()

['Taxonomy',
 'Body - Training',
 'Default',
 'Master Deck',
 'zz Input',
 'zz Input 2',
 'zz_Temp',
 'Testing']

In [17]:
mb.anki_find_notes('Master Deck')[:3]

['1517497423118', '1517497467024', '1517497482074']

In [18]:
id_ = mb.anki_add_note(deck='Testing', front='Question', back='Answer')
print(id_)

1560028646047


In [19]:
mb.anki_get_note(id_)

('Question', 'Answer')

In [20]:
mb.anki_update_note(id_, front='Question2', back='Answer2')

In [21]:
mb.anki_get_note(id_)

('Question2', 'Answer2')

In [22]:
mb.anki_delete_note(id_)

In [23]:
try:
    mb.anki_get_note(id_)
except KeyError:
    print('Note does not exist')

Note does not exist


# Jupyter -> Anki

In [24]:
class Command:
    def __init__(self, cmd, id_, head, body, deck=None, cell=None):
        self.cmd = cmd
        self.id = id_
        self.head = head
        self.body = body
        self.deck = deck
        self.cell = cell

In [25]:
def figure_out_command(meta, head, body, node_ids):
    if 'id' not in meta:
        # This note has empty <!---->, meaning it was just added in Jupyter
        cmd = Command('add', None, head, body)
    else:
        # Get note Anki ID
        id_ = meta['id']

        if id_ not in note_ids:
            # card was probably manually deleted from Anki, recreate it
            cmd = Command('add2', None, head, body)
        else:
            # ID exists in database

            front, back = mb.anki_get_note(id_)
            if front != head or back != body:
                cmd = Command('update', id_, head, body)
            else:
                cmd = None
    
    return cmd

In [26]:
def execute_command(cmd):
    assert cmd.deck is not None
    assert cmd.cell is not None
    if cmd.cmd in ['add', 'add2']:
        id_ = mb.anki_add_note(cmd.deck, cmd.head, cmd.body)
        new_meta = mb.put_meta(cmd.cell.source, id_)
        cmd.cell.source = new_meta
    elif cmd.cmd == 'update':
        mb.anki_update_note(cmd.id, cmd.head, cmd.body)
    else:
        raise ValueError(f'Unknown command: {cmd.cmd}')

In [28]:
with open('Example.ipynb', 'r') as f:
    nb = nbformat.read(f, as_version=4)

In [29]:
note_ids = mb.anki_find_notes('Testing')
print(note_ids)

['1560026882572', '1560026908772']


In [30]:
commands = []

for cell in nb['cells']:
    if not mb.is_flashcard(cell):
        continue
        
    meta, head, body = mb.process_cell_source(cell.source)
    cmd = figure_out_command(meta, head, body, note_ids)
    if cmd is not None:
        cmd.deck = 'Testing'
        cmd.cell = cell
        commands.append(cmd)

In [31]:
[c.cmd + ': ' + c.head for c in commands]

['add: Example question goes here', 'add: Another question goes here']

In [33]:
for cmd in commands:
    execute_command(cmd)

In [34]:
with open('Example.ipynb', 'w') as f:
    nbformat.write(nb, f)